In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, TaskType, get_peft_model
from trl import SFTTrainer
import warnings
import numpy as np
warnings.filterwarnings('ignore')

2024-05-13 09:13:03.003010: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 09:13:03.056025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 09:13:03.056074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 09:13:03.060943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 09:13:03.079527: I tensorflow/core/platform/cpu_feature_guar

In [3]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"


# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1


# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./summary_results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-1

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.01

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [4]:
import pandas as pd
# Load your DataFrame
data = pd.read_csv('summaries.csv')

In [5]:
data.head()

,article_id,cleaned_article,summary
0,4_1899-01-03_p4_sn86090500_00513687990_1899010...,only one or two men are holding out against gr...,The article discusses the importance of gravel...
1,2_1899-01-03_p4_sn85066387_00175037810_1899010...,call headquarters sacra mento jan 2 the organi...,The assembly in Sacramento completed its organ...
2,1_1899-01-03_p4_sn85066387_00175037810_1899010...,which are preserved in the treaty of peace wil...,The article discusses the resolutions adopted ...
3,31_1899-01-03_p2_sn89051168_00393343084_189901...,shermans ample supply of whisky george a sheri...,The article describes an encounter between Geo...
4,1_1899-01-03_p12_sn83045462_00280655247_189901...,messrs frank p noyes and william frizzell rep...,"Messrs Frank P. Noyes and William Frizzell, re..."


In [6]:
import pandas as pd

# Assuming 'data' is your DataFrame
data.drop('article_id', axis=1, inplace=True)

In [7]:
data

,cleaned_article,summary
0,only one or two men are holding out against gr...,The article discusses the importance of gravel...
1,call headquarters sacra mento jan 2 the organi...,The assembly in Sacramento completed its organ...
2,which are preserved in the treaty of peace wil...,The article discusses the resolutions adopted ...
3,shermans ample supply of whisky george a sheri...,The article describes an encounter between Geo...
4,messrs frank p noyes and william frizzell rep...,"Messrs Frank P. Noyes and William Frizzell, re..."
...,...,...
8995,best instruction in antique and life drawing s...,The article describes luxurious and meticulous...
8996,for sale nust be soid one of those nsw s ru0m ...,The article is a historical advertisement for ...
8997,canadian pacific railway intended steamship sa...,The Canadian Pacific Railway announced intende...
8998,dents and many dead infants claimed his attent...,"The article discusses the various homicides, a..."


In [8]:
import json
import re
from pprint import pprint
 
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer
 
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

In [9]:
import pandas as pd

# Load your CSV file
data = pd.read_csv('summaries.csv')

# Remove the 'article_id' column
data.drop('article_id', axis=1, inplace=True)


In [10]:
# Assuming 'cleaned_article' is your Input and 'summary' is your Response

transformed_data = pd.DataFrame({
    'text': data.apply(lambda row: f"### Instruction:\n\nBelow is a conversation between a human and an AI agent. Write a summary of the conversation.\n\n### Input:\n\n{row['cleaned_article']}\n\n### Response:\n\n{row['summary']}", axis=1)
})
# Save the transformed data to a new CSV or text file
# transformed_data.to_csv('transformed_dataset.csv', index=False, header=["Conversation"])


In [11]:
transformed_data.head()

,text
0,### Instruction:\n\nBelow is a conversation be...
1,### Instruction:\n\nBelow is a conversation be...
2,### Instruction:\n\nBelow is a conversation be...
3,### Instruction:\n\nBelow is a conversation be...
4,### Instruction:\n\nBelow is a conversation be...


In [12]:
notebook_login()

In [13]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
 
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )
 
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
 
    return model, tokenizer

In [14]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

transformed_train_df, transformed_test_val_df = train_test_split(transformed_data, test_size=0.3, random_state=42)
# Create a Dataset object with the DataFrame

transformed_train_dataset = Dataset.from_pandas(transformed_train_df)
transformed_val_dataset = Dataset.from_pandas(transformed_test_val_df)

In [15]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [17]:
from transformers import TrainerCallback

class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience):
        super().__init__()
        self.early_stopping_patience = early_stopping_patience
        self.best_eval_metric = float("-inf")
        self.num_bad_epochs = 0

    def on_evaluate(self, args, state, control, **kwargs):
        eval_metric = state.log_history[-1].get("eval_metric", None)  # Replace "eval_metric" with your actual evaluation metric key
        if eval_metric is not None:
            if eval_metric > self.best_eval_metric:
                self.best_eval_metric = eval_metric
                self.num_bad_epochs = 0
            else:
                self.num_bad_epochs += 1
                if self.num_bad_epochs >= self.early_stopping_patience:
                    control.should_training_stop = True

# Usage example


In [18]:
# from transformers.integrations import EarlyStoppingCallback

lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]
 
 
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir="summary",
    # report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
)

In [19]:
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_train_dataset,
    eval_dataset=transformed_val_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],  # Adjust patience as needed
)


Map:   0%|          | 0/6300 [00:00<?, ? examples/s]

Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [20]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=2,  # Reduced batch size
    per_device_eval_batch_size=2,  # Reduced eval batch size
    gradient_accumulation_steps=2,  # Reduced accumulation steps
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,  # Lower learning rate
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir="summary",
    lr_scheduler_type="cosine",
    gradient_checkpointing=False,  # Disable gradient checkpointing
    # max_seq_length=128,  # Reduced sequence length
)

trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_train_dataset,
    eval_dataset=transformed_val_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=128,  # Reduced sequence length
    tokenizer=tokenizer,
    args=training_arguments,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],  # Adjust patience as needed
)


Map:   0%|          | 0/6300 [00:00<?, ? examples/s]

Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [21]:
trainer.train()

wandb: Currently logged in as: pavanvicky2015 (sjsuedu). Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_CUDA_mm)

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fb230126cd0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fb162391250, execution_count=21 error_before_exec=None error_in_exec=Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_CUDA_mm) info=<ExecutionInfo object at 7fb22818f3d0, raw_cell="trainer.train()" store_history=True silent=False shell_futures=True cell_id=30692297-dd8f-4293-be6b-e07e68869398> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [ ]:
import wandb
trainer.model.save_pretrained("Llama2_7B_PEFT_Summaries")

wandb.finish()